In [ ]:
# .pbファイル読み込み
tf_model_path = './stylize_quantized.pb'

with open(tf_model_path, 'rb') as f:
    serialized = f.read()

In [ ]:
# ファイルからグラフ定義を取得
import tensorflow as tf

tf.reset_default_graph()
gdef = tf.GraphDef()
gdef.ParseFromString(serialized)

In [ ]:
# 入力テンソルの名前を取得する
with tf.Graph().as_default() as g:
    tf.import_graph_def(gdef, name='')
    ops = g.get_operations()
    
    # プレースホルダの出力名を調べる
    N = len(ops)
    for i in range(N):
        if ops[i].type == 'Placeholder':
            for x in ops[i].outputs:
                print("output name = {}, shape: {},"
                      .format(x.name, x.get_shape())),

In [ ]:
# 出力テンソルの名前を取得する
with tf.Graph().as_default() as g:
    tf.import_graph_def(gdef, name='')
    ops = g.get_operations()

    # 最後の10ノードの入出力を調べる
    N = len(ops)
    for i in range(N-10,N):
        print('\n\nop id {} : op type: "{}"'.format(str(i), ops[i].type));
        print('input(s):'),
        for x in ops[i].inputs:
            print("name = {}, shape: {}, ".format(x.name, x.get_shape())),
        print('\noutput(s):'),
        for x in ops[i].outputs:
            print("name = {}, shape: {},".format(x.name, x.get_shape())),

In [ ]:
# Core MLモデルへ変換する
import tfcoreml

mlmodel = tfcoreml.convert(
        tf_model_path = tf_model_path,
        mlmodel_path = './stylize_1.mlmodel',
        output_feature_names = ['Squeeze:0'],
        input_name_shape_dict = {'input:0':[1,256,256,3], 'style_num:0':[26]},
        image_input_names = ['input:0'])

In [ ]:
print(mlmodel)

In [ ]:
# Specを取得
spec = mlmodel.get_spec()
print(spec.description)

In [ ]:
output = spec.description.output[0]
print(output.name)

In [ ]:
# 出力の型を変更する
import coremltools
from coremltools.proto import FeatureTypes_pb2 as ft

output.type.imageType.colorSpace = ft.ImageFeatureType.ColorSpace.Value('RGB')
output.type.imageType.width = 256
output.type.imageType.height = 256

In [ ]:
print(spec.description)

In [ ]:
# .mlmodelファイルとして保存
coremltools.models.utils.save_spec(spec, './stylize.mlmodel')